### 청약 매물 기사 크롤링 함수

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm.notebook import tqdm

1. 검색 URL 생성 함수 (아파트명 + 청약)

In [3]:
import urllib.parse

def generate_news_url(apartment_name: str) -> str:
    """아파트명 + 청약 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    query = f'"{apartment_name}" 청약'  # 쌍따옴표 포함 검색
    
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "1"  # 최신순 정렬
    }
    
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

2. 네이버 뉴스 전용 크롤링 함수

In [4]:
import requests
from bs4 import BeautifulSoup

def crawl_naver_news(url: str, max_articles: int = 10) -> list:
    """네이버 뉴스 기사 전문 크롤링"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # 검색 결과 페이지 파싱
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 네이버 뉴스 링크 필터링
    news_links = []
    for a in soup.select('a.info'):
        if '네이버뉴스' in a.text and 'news.naver.com' in a['href']:
            news_links.append(a['href'])
            if len(news_links) >= max_articles:
                break
    
    # 기사 본문 추출
    articles = []
    for article_url in news_links:
        try:
            article_res = requests.get(article_url, headers=headers, timeout=5)
            article_soup = BeautifulSoup(article_res.text, 'html.parser')
            
            # 표준화된 선택자 사용
            title = article_soup.select_one('#title_area').get_text(strip=True)
            date = article_soup.select_one('.media_end_head_info_datestamp_time')['data-date-time']
            content = article_soup.select_one('#newsct_article, #dic_area').get_text(strip=True)
            
            articles.append({
                "title": title,
                "date": date,
                "content": content[:500] + "...",  # 500자 요약
                "url": article_url
            })
            
        except Exception as e:
            print(f"기사 처리 실패 ({article_url}): {str(e)}")
    
    return articles

3. 통합 실행

In [5]:
if __name__ == "__main__":
    # 아파트명 입력
    apartment_name = "래미안 원베일리"
    
    # 1단계: 검색 URL 생성
    search_url = generate_news_url(apartment_name)
    print(f"생성된 검색 URL: {search_url}")
    
    # 2단계: 네이버 뉴스 기사 크롤링
    news_data = crawl_naver_news(search_url)
    
    # 결과 출력
    print(f"\n{apartment_name} 관련 청약 뉴스 {len(news_data)}건")
    for idx, article in enumerate(news_data, 1):
        print(f"\n[{idx}] {article['title']}")
        print(f"작성일: {article['date']}")
        print(f"요약: {article['content']}")
        print(f"URL: {article['url']}")

생성된 검색 URL: https://search.naver.com/search.naver?where=news&query=%22%EB%9E%98%EB%AF%B8%EC%95%88%20%EC%9B%90%EB%B2%A0%EC%9D%BC%EB%A6%AC%22%20%EC%B2%AD%EC%95%BD&sm=tab_opt&sort=1

래미안 원베일리 관련 청약 뉴스 4건

[1] 원베일리 전용 84㎡ 70억 원에 거래?…정부 규제 나올까
작성일: 2025-03-07 09:13:55
요약: ▲ 서초구 반포동 '래미안 원베일리' 아파트 모습현재 강남에선 서울 서초구 반포동 래미안 원베일리의 한강변 전용면적 84㎡가 70억 원에 매매가 됐느냐, 아니냐를 놓고 중개업계가 시끌시끌합니다.이 계약이 성사됐다면 지난해 8월 초 60억 원에 팔린 뒤 불과 7개월 만에 약 10억 원이 오른 것이기 때문입니다.3.3㎡당 가격이 2억 원이 넘는 역대 최고가입니다.현재 포털사이트에 올라온 원베일리 한강변 84㎡ 매물의 최고 시세는 72억 원에 달합니다.반포동의 한 중개업소 대표는 "아직 실거래가 신고가 되지 않아 실제 계약 여부는 확인할 수 없지만 70억 원에 팔렸다는 말도 있고 1억 원 낮은 69억 원에 거래됐다는 설도 있다"며 "진짜 팔렸다면 한강 조망 아파트의 희소가치를 높게 본 것 같다"고 말했습니다.최근 강남 '하이엔드급' 아파트 가격의 상승세가 거침없습니다.살 사람은 있는데 매물이 없어서 못팔고, 가격이 아무리 높아도 희소가치가 있으면 거래되는 시장입니다.반포의 대장주 아크로리...
URL: https://n.news.naver.com/mnews/article/055/0001237786?sid=101

[2] [서미숙의 집수다] 원베일리 전용 84㎡ 70억원에 거래?…정부 규제 나올까
작성일: 2025-03-07 08:57:21
요약: 강남 초고가 아파트 고공행진에 토허제 해제 단지도 "5년 억눌렸던 가격 올려야"2월 계약 신고 건수 3천603건, 1월 거래량 뛰어넘어…외곽도 급매 소